<a href="https://colab.research.google.com/github/nicolashg7/AI/blob/main/AI_Data_Documents.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install transformers
!pip install openai
from google.colab import drive
from transformers import pipeline
import openai

drive.mount('/content/drive')
path = ""
openai.api_key = ""

# Load chatGPT
def ask_chatgpt(prompt):
    response = openai.chat.completions.create(
        model="gpt-3.5-turbo-0125",
        messages=[
            {"role": "user", "content": prompt}
        ]
    )
    return response.choices[0].message.content

# Load a file
def load_file(path):
  with open(path, 'r') as file:
    text = file.read()
  return text



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.9/386.9 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.2/325.2 kB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.2 MB/s eta 0:00:00
Mounted at /content/drive


In [ ]:
# Mount files

files = ["cape_town.txt", "madrid.txt", "rio_de_janeiro.txt", "sydney.txt", "tokyo.txt"]

# Load the model with GPU
pipe = pipeline(task = "text-classification", model = "classla/multilingual-IPTC-news-topic-classifier", device = 0)

for text_file in files:
  # Read the file
  f = open(path + text_file, "r")
  text = f.read()
  f.close()

  # Run the model
  classified = pipe(text)
  relevancy = "relevant" if classified[0]['label'] == 'lifestyle and leisure' else "irrelevant"

  print(f"The text {text_file} is {relevancy}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
The text cape_town.txt is relevant
The text madrid.txt is irrelevant
The text rio_de_janeiro.txt is relevant
The text sydney.txt is relevant
The text tokyo.txt is relevant


In [ ]:
files = ["cape_town.txt"]

# Produce html
for text_file in files:
  # Read the file
  f = path + text_file
  with open(f,'r') as file:
    text = file.read()

  # Run the model
  prompt = f"""
    Produce a html text for the following text: {text}
    Highlight in pink the restaurant names and in green the dishes.
  """
  html = ask_chatgpt(prompt)
  # Write the answer in a new file
  file_path = path + text_file[:-4] + '.html'
  with open(file_path, 'w') as file:
    file.write(html)


In [ ]:
import csv
from IPython.display import display, Markdown

file_path = path + 'itinerary.csv'

with open(file_path, "r") as file:
  itinerary = file.read()

dic_itinerary = []
for row in csv.DictReader(itinerary.splitlines()):
  dic_itinerary.append(row)


prompt = f""" I will visit {dic_itinerary[2]["City"]} in {dic_itinerary[2]["Country"]},
from {dic_itinerary[2]["Arrival"]} to {dic_itinerary[2]["Departure"]}.
Please, provide a detailed itinerary for this visit in Markdown format. Do not include any other text at the end.
"""

response = ask_chatgpt(prompt)
display(Markdown(response))


# Cape Town Itinerary

## Day 1: 17-Jul
- Arrival at Cape Town International Airport
- Check into hotel
- Visit Victoria & Alfred Waterfront for dinner and shopping

## Day 2: 18-Jul
- Explore Table Mountain National Park
- Take a cable car up Table Mountain
- Visit Kirstenbosch Botanical Gardens

## Day 3: 19-Jul
- Wine tasting tour in Stellenbosch or Franschhoek
- Visit Groot Constantia or another winery
- Enjoy lunch at a vineyard

## Day 4: 20-Jul
- Visit Robben Island
- Explore the District Six Museum
- Dinner at a local restaurant

## Day 5: 21-Jul
- Shark cage diving in Gansbaai
- Visit Hermanus for whale watching
- Dinner at a seafood restaurant

## Day 6: 22-Jul
- Explore Boulders Beach
- Visit Cape Point and Cape of Good Hope
- Drive along Chapman's Peak Drive

## Day 7: 23-Jul
- Explore the Bo-Kaap neighborhood
- Visit the Two Oceans Aquarium
- Enjoy dinner at a local seafood restaurant

## Day 8: 24-Jul
- Check out of hotel
- Visit Greenmarket Square for souvenirs
- Depart from Cape Town International Airport

In [15]:
import csv

files_names = ["itinerary.csv","cape_town.txt", "rio_de_janeiro.txt", "sydney.txt", "tokyo.txt"]
name_cities = ["Cape Town", "Rio de Janeiro", "Sydney", "Tokyo"]

#Load files
files_content = []
for file in files_names:
    content = load_file(path + file)
    files_content.append(content)

itinerary = []
for row in csv.DictReader(files_content[0].splitlines()):
  itinerary.append(row)

files_content = files_content[1:]

dict_resto = {}
restaurants = {}
for content, name in zip(files_content, name_cities):
  prompt = f"""Extract a list with the name of the restaurants and the most famous dish
  for each of restaurant in the following text: {content}.
  Provide your answer in CSV format, ready to save.
  Exclude the csv declaration, don't add spaces after the comma, do not include column headers.
  Format:
  Res_1, Dish_1
  Res_2, Dish_2
  ...
  """
  response = ask_chatgpt(prompt)

  for row in csv.reader(response.splitlines()):
    restaurant, dish = row  # Unpack each row
    restaurants[restaurant.strip()] = dish.strip()  # Use restaurant as key and dish as value

  name = name.replace(".txt", "")
  dict_resto[name] = restaurants
  restaurants = {}

travel_itinerary = {}
for travel_spot in itinerary:

  prompt = f"""I will travel to {travel_spot['City']} in {travel_spot['Country']},
  from {travel_spot['Arrival']} to {travel_spot['Departure']}.
  Please, provide a detailed itinerary for this visit in Markdown format. Do not include any other text at the end.
  For each day provide a spot for having lunch and for having dinner and choose the restaurant from the following list:
  {dict_resto[travel_spot['City']]}. Do not repeate the same restaurant in any of the days and include
  the dish for each restaurant.
  """
  travel_itinerary[travel_spot['City']] = ask_chatgpt(prompt)


In [17]:
from IPython.display import display, Markdown
display(Markdown(travel_itinerary['Tokyo']))

# Tokyo Itinerary

## Day 1: 10-Aug
- **Morning:** Explore Asakusa and visit Senso-ji Temple
- **Lunch:** Tsukiji Outer Market (Fresh sashimi and street food)
- **Afternoon:** Visit Tokyo Skytree
- **Dinner:** Narisawa (Innovative tasting menu)

## Day 2: 11-Aug
- **Morning:** Visit Meiji Shrine
- **Lunch:** Ichiran Ramen (Tonkotsu ramen)
- **Afternoon:** Explore Harajuku and Takeshita Street
- **Dinner:** Ginza Kojyu (Kaiseki)

## Day 3: 12-Aug
- **Morning:** Visit Tsukiji Fish Market
- **Lunch:** Sukiyabashi Jiro (Omakase sushi)
- **Afternoon:** Explore Odaiba and visit TeamLab Borderless
- **Dinner:** Akasaka Kikunoi (Kaiseki)

## Day 4: 13-Aug
- **Morning:** Visit Imperial Palace
- **Lunch:** Ginza Kojyu (Kaiseki)
- **Afternoon:** Explore Shibuya and cross the Shibuya Crossing
- **Dinner:** Tsukiji Outer Market (Fresh sashimi and street food)

## Day 5: 14-Aug
- **Morning:** Visit Ueno Park and Ueno Zoo
- **Lunch:** Akasaka Kikunoi (Kaiseki)
- **Afternoon:** Explore Akihabara
- **Dinner:** Sukiyabashi Jiro (Omakase sushi)

## Day 6: 15-Aug
- **Morning:** Visit Tokyo Disneyland or DisneySea
- **Lunch:** Narisawa (Innovative tasting menu)
- **Afternoon:** Explore Shinjuku and visit the Tokyo Metropolitan Government Building
- **Dinner:** Ichiran Ramen (Tonkotsu ramen)

## Day 7: 16-Aug
- **Morning:** Visit Roppongi Hills
- **Lunch:** Tsukiji Outer Market (Fresh sashimi and street food)
- **Afternoon:** Stroll around Omoide Yokocho
- **Dinner:** Ginza Kojyu (Kaiseki)

## Day 8: 17-Aug
- **Morning:** Visit Yoyogi Park
- **Lunch:** Akasaka Kikunoi (Kaiseki)
- **Afternoon:** Last-minute shopping in Shibuya
- **Dinner:** Narisawa (Innovative tasting menu)